In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116 -q

In [ ]:
!pip3 install ultralytics -q

In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.53 🚀 Python-3.9.13 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11020MiB)
Setup complete ✅ (48 CPUs, 125.7 GB RAM, 359.2/456.9 GB disk)


In [ ]:
from ultralytics import YOLO
import pandas as pd

In [ ]:
#folder names with the model to validate
folders = ['Adam_Aligned_4', 'Adam_Default_4', 'Adam_NO_4', 'SGD_Aligned_4', 'SGD_Default_4', 'SGD_NO_4']
folder = folders[0]

#splits to use for validation
splits = ['test', 'train', 'val']
split = splits[0]

#all categories in the data set (yolo order)
categories = ["cup", "bowl", "bottle", "plate", "pen", "book", "stapler", "hole_puncher", "flashdrive", "smartphone", "all"]

Generate CSV with mAP values

In [ ]:
def get_csv(result):
    p = []
    r = []
    mAP50 = []
    mAP75 = []
    mAP50_95 = []


    for i in range(len(categories)):
        if i == len(categories)-1:
            p.append(result.box.mp)
            r.append(result.box.mr)
            mAP50.append(result.box.map50)
            mAP75.append(result.box.map75)
            mAP50_95.append(result.box.map)
        else:
            res = result.class_result(i)
            p.append(res[0])
            r.append(res[1])
            mAP50.append(res[2])
            mAP75.append(" ")
            mAP50_95.append(res[3])

    df = pd.DataFrame(list(zip(categories, p, r, mAP50, mAP75, mAP50_95)))
    df.columns = ["Class", "P", "R", "mAP50", "mAP75", "mAP50:95"]
    df.to_csv("/home/jovyan/" + folder + "/" + split + "_val_data.csv")
    #df.to_csv("/home/jovyan/" + folder + "/foreign_val_data.csv")

Itterate over all models

In [ ]:
# Load a model
for u in range(len(folders)):
    model = YOLO("/home/jovyan/" + folder + "/train/weights/best.pt") 
    folder = folders[u]
    for i in range(len(splits)):
        split = splits[i]
        name = 'val_' + 'foreign' #split 
        results = model.val(split=split, name=name, project=folder)  
        get_csv(results)